# Exercise Set 7: Data structuring  3

*Morning, August 15, Morning*

In this Exercise Set we finalize our work with the weather data we started working on in Exercise Set 4. We will also study a dataset of traffic data from Copenhagen to iterate through the pandas workflow once more. 

In [131]:
%matplotlib inline 

import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib as mpl

# Increases the plot size a little
mpl.rcParams['figure.figsize'] = 11, 6


## Exercise Section 7.1: Weather data, part 3
We continue with the final part of three exercises on structuring weather data. In this exercise you must use the function for fetching and structuring weather data which you made in Exercise 6.1.5.

> **Ex. 7.1.1:** Plot the monthly max,min, mean, first and third quartiles for maximum temperature for our station with the ID _'ITE00100550'_ in 1864. 

> *Hint*: the method `describe` computes all these measures.

In [141]:
# [Answer to Ex. 7.1.1]

import requests 
import pandas as pd

url = 'https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/1864.csv.gz'

df_weather = pd.read_csv(url,
                         compression='gzip',
                         header=None).iloc[:,:4]

df_weather.columns = ['station', 'datetime', 'obs_type', 'obs_value']
df_weather['obs_value'] = df_weather['obs_value'] / 10
df_select = df_weather[(df_weather.station == 'ITE00100550') & (df_weather.obs_type == 'TMAX')].copy()
df_select['TMAX_F'] = 32 + 1.8 * df_select['obs_value']
df_sorted = df_select.reset_index(drop=True).sort_values(by=['obs_value'])
df_sorted["datetime_dt"] = pd.to_datetime(df_sorted['datetime'], format="%Y%m%d")
df_sorted['month'] = df_sorted.datetime_dt.dt.month
#df_sorted.head()
df_sorted.describe()





,datetime,obs_value,TMAX_F,month
count,3.660000e+02,366.000000,366.000000,366.000000
mean,1.864067e+07,16.249727,61.249508,6.513661
std,3.457653e+02,10.158772,18.285789,3.455958
min,1.864010e+07,-6.300000,20.660000,1.000000
25%,1.864040e+07,7.600000,45.680000,4.000000
50%,1.864070e+07,16.150000,61.070000,7.000000
75%,1.864098e+07,25.325000,77.585000,9.750000
max,1.864123e+07,34.800000,94.640000,12.000000


In [129]:
def b(year):
    url = 'https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/{y}.csv.gz'.format(y=year)

    df_weather = pd.read_csv(url,
                             compression='gzip',
                             header=None).iloc[:,:4]

    df_weather.columns = ['station', 'datetime', 'obs_type', 'obs_value']
    df_weather['obs_value'] = df_weather['obs_value'] / 10
    df_select = df_weather[(df_weather.obs_type == 'TMAX')].copy()
    df_select['TMAX_F'] = 32 + 1.8 * df_select['obs_value']
    df_sorted = df_select.reset_index(drop=True).sort_values(by=['obs_value'])
    df_sorted["datetime_dt"] = pd.to_datetime(df_sorted['datetime'], format="%Y%m%d")
    df_sorted['month'] = df_sorted.datetime_dt.dt.month
    return df_sorted

#df_sorted.describe()

    

In [135]:
b(1864).describe()

,datetime,obs_value,TMAX_F,month
count,5.686000e+03,5686.000000,5686.000000,5686.000000
mean,1.864067e+07,13.546711,56.384080,6.576504
std,3.431089e+02,9.312093,16.761768,3.429404
min,1.864010e+07,-34.000000,-29.200000,1.000000
25%,1.864040e+07,7.500000,45.500000,4.000000
50%,1.864070e+07,14.800000,58.640000,7.000000
75%,1.864100e+07,20.100000,68.180000,10.000000
max,1.864123e+07,35.900000,96.620000,12.000000


> **Ex. 7.1.2:** Get the processed data from years 1864-1867 as a list of DataFrames. Convert the list into a single DataFrame by concatenating vertically. 

In [139]:
# [Answer to Ex. 7.1.2]
ITE_station1864 = b(1864)
ITE_station1865 = b(1865)
ITE_station1866 = b(1866)
ITE_station1867 = b(1867)


ITE_st_all = [ITE_station1864, ITE_station1865, ITE_station1866 , ITE_station1867]
#print(ITE_st_all)

concatITE = pd.concat(ITE_st_all, axis=0, join="outer", sort=False )

print(concatITE)

          station  datetime obs_type  obs_value  TMAX_F datetime_dt  month
845   SZ000006717  18640226     TMAX      -34.0  -29.20  1864-02-26      2
577   SZ000006717  18640208     TMAX      -17.4    0.68  1864-02-08      2
699   CA006158350  18640217     TMAX      -16.7    1.94  1864-02-17      2
42    SZ000006717  18640103     TMAX      -16.5    2.30  1864-01-03      1
16    CA006158350  18640102     TMAX      -16.1    3.02  1864-01-02      1
...           ...       ...      ...        ...     ...         ...    ...
47    ASN00074128  18670102     TMAX       42.8  109.04  1867-01-02      1
9310  ASN00074128  18671220     TMAX       43.3  109.94  1867-12-20     12
375   ASN00074128  18670115     TMAX       45.1  113.18  1867-01-15      1
73    ASN00074128  18670103     TMAX       46.2  115.16  1867-01-03      1
349   ASN00074128  18670114     TMAX       46.7  116.06  1867-01-14      1

[30003 rows x 7 columns]


> **Ex. 7.1.3:** Parse the station location data which you can find at https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt. Merge station locations onto the weather data spanning 1864-1867.  

> _Hint:_ The location data have the folllowing format, 

```
------------------------------
Variable   Columns   Type
------------------------------
ID            1-11   Character
LATITUDE     13-20   Real
LONGITUDE    22-30   Real
ELEVATION    32-37   Real
STATE        39-40   Character
NAME         42-71   Character
GSN FLAG     73-75   Character
HCN/CRN FLAG 77-79   Character
WMO ID       81-85   Character
------------------------------
```

> *Hint*: The station information has fixed width format - does there exist a pandas reader for that?

In [144]:
# [Answer to Ex. 7.1.3]
url = 'https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt'

col = ['station', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'STATE', 'NAME', 'GSN FLAG', 'HCN/CRN FLAG', 'WMO ID']
#make and save DataFrame for new URL
new = pd.read_fwf(url, names=col) #fwf = fixed width format

#station and ID=station
col2 = [ITE_st_all, new]

pd.merge(concatITE, new, on='station', how='inner')

,station,datetime,obs_type,obs_value,TMAX_F,datetime_dt,month,LATITUDE,LONGITUDE,ELEVATION,STATE,NAME,GSN FLAG,HCN/CRN FLAG,WMO ID
0,SZ000006717,18640226,TMAX,-34.0,-29.20,1864-02-26,2,45.8667,7.1667,2472.0,COL DU GRAND ST-BERNARD,NaN,GSN,6717.0,NaN
1,SZ000006717,18640208,TMAX,-17.4,0.68,1864-02-08,2,45.8667,7.1667,2472.0,COL DU GRAND ST-BERNARD,NaN,GSN,6717.0,NaN
2,SZ000006717,18640103,TMAX,-16.5,2.30,1864-01-03,1,45.8667,7.1667,2472.0,COL DU GRAND ST-BERNARD,NaN,GSN,6717.0,NaN
3,SZ000006717,18640210,TMAX,-15.9,3.38,1864-02-10,2,45.8667,7.1667,2472.0,COL DU GRAND ST-BERNARD,NaN,GSN,6717.0,NaN
4,SZ000006717,18640209,TMAX,-15.0,5.00,1864-02-09,2,45.8667,7.1667,2472.0,COL DU GRAND ST-BERNARD,NaN,GSN,6717.0,NaN
5,SZ000006717,18640219,TMAX,-14.7,5.54,1864-02-19,2,45.8667,7.1667,2472.0,COL DU GRAND ST-BERNARD,NaN,GSN,6717.0,NaN
6,SZ000006717,18640205,TMAX,-14.4,6.08,1864-02-05,2,45.8667,7.1667,2472.0,COL DU GRAND ST-BERNARD,NaN,GSN,6717.0,NaN
7,SZ000006717,18640220,TMAX,-14.2,6.44,1864-02-20,2,45.8667,7.1667,2472.0,COL DU GRAND ST-BERNARD,NaN,GSN,6717.0,NaN
8,SZ000006717,18640207,TMAX,-13.8,7.16,1864-02-07,2,45.8667,7.1667,2472.0,COL DU GRAND ST-BERNARD,NaN,GSN,6717.0,NaN
9,SZ000006717,18640218,TMAX,-13.3,8.06,1864-02-18,2,45.8667,7.1667,2472.0,COL DU GRAND ST-BERNARD,NaN,GSN,6717.0,NaN


## Exercise Section 7.2: Traffic data in Copenhagen

In this second part of exercise set 7 you will be working with traffic data from Copenhagen Municipality.

The municipality have made the data openly available through the [opendata.dk](http://www.opendata.dk/) platform. We will use the data from traffic counters to construct a dataset of hourly traffic. We will use this data to get basic insights on the development in traffic over time and relate it to weather. The gist here is to practice a very important skill in Data Science: being able to quickly fetch data from the web and structure it so that you can work with it. Scraping usually gets a bit more advanced than what we will do today, but the following exercises should give you a taste for how it works. The bulk of these exercise, however, revolve around using the Pandas library to structure and analyze data.

### Part 7.2.a: getting some data to work with

Hence follows a simple scraping exercise where you (1) collect urls for datasets in the webpage listing data on traffic counters and (2) use these urls to load the data into one dataframe.

> **Ex. 7.2.1:** Using the requests module, extract the html markup of the webpage data.kk.dk/dataset/faste-trafiktaellinger and store it as a string in a new variable.

In [ ]:
# [Answer to Ex. 7.2.1]


> **Ex. 7.2.2:** Using the re module, extract a list of all the urls in the html string and store them in a new variable.

> _Hint:_ Try using the re.findall method. You may want to Google around to figure out how to do this. Protip: searching for something along the lines of "extract all links in html regex python" and hitting the first StackOverflow link will probably get you farther than reading elaborate documentation.

In [ ]:
# [Answer to Ex. 7.2.2]

> **Ex. 7.2.3:** Create a new variable that only contains the links that point to downloadable traffic data sheets. 

> _Hint:_ You want to filter the results from above. For example to only include urls with the term 'download' in them.

In [34]:
# [Answer to Ex. 7.2.3]

> **Ex. 7.2.4:** Using pd.read_excel method, load the datasets into a list. Your resulting variable should hold a list of Pandas dataframes.

> _Hint:_ you may want to set the skiprows= keyword argument. 

In [ ]:
# [Answer to Ex. 7.2.4]

> **Ex. 7.2.5:** Merge the list of dataframes into a single dataframe.

In [ ]:
# [Answer to Ex. 7.2.5]

### Part 7.2.b Structuring your data

If you successfully completed the previous part, you should now have a dataframe with about 183.397 rows (if your number of rows is close but not the same, worry not—it matters little in the following). Well done! But the data is still in no shape for analysis, so we must clean it up a little.

161.236 rows (and 30 columns) is a lot of data. ~3.3 MB by my back-of-the-envelope calculations, so not "Big Data", but still enough to make your CPU heat up if you don't use it carefully. Pandas is built to handle fairly large dataframes and has advanced functionality to perform very fast operations even when the size of your data grows huge. So instead of working with basic Python we recommend working pandas built-in procedures as they are constructed to be fast on dataframes.

Nerd fact: the reason pandas is much faster than pure Python is that dataframes access a lower level programming languages (namely C, C++) which are multiple times faster than Python. The reason it is faster is that it has a higher level of explicitness and thus is more difficult to learn and navigate.

> **Ex. 7.2.6:** Reset the row indices of your dataframe so the first index is 0 and the last is whatever the number of rows your dataframe has. Also drop the column named 'index' and the one named `Spor`.

In [ ]:
# [Answer to Ex. 7.2.6]

> **Ex. 7.2.7:** Rename variables from Danish to English using the dictionary below.

```python 
dk_to_uk = {
    'Vejnavn':'road_name',
    '(UTM32)':'UTM32_north',
    '(UTM32).1':'UTM32_east',
    'Dato':'date',
    'Vej-Id':'road_id'
}
```

In [ ]:
# [Answer to Ex. 7.2.7]

Python is quite efficient. For example, when you create a new dataframe by manipulating an old one, Python notices that—apart from some minor changes—these two objects are almost the same. Since memory is a precious resource, Python will represent the values in the new dataframe as references to the variables in the old dataset. This is great for performance, but if you for whatever reason change some of the values in your old dataframe, values in the new one will also change—and we don't want that! Luckily, we can break this dependency.

> **Ex. 7.2.8:** Break the dependencies of the dataframe that resulted from Ex. 7.2.7 using the `.copy` method. Delete all other dataframes.

In [ ]:
# [Answer to Ex. 7.2.8]

Once you have structured appropriately, something that you will want to do again and again is selecting subsets of the data. Specifically, it means that you select specific rows in the dataset based on some column values.

>**Ex. 7.2.9:** Create a new column in the dataframe called total that is True when the last letter of road_id is T and otherwise False.

> _Hint:_ you will need the `pd.Series.str` attribute for this.

In [ ]:
# [Answer to Ex. 7.2.9]

> **Ex. 7.2.10:** Select rows where total is True. Delete all the remaining observations.

In [ ]:
# [Answer to Ex. 7.2.10]

> **Ex. 7.2.11:** Make two datasets based on the lists of columns below. Call the dataset with spatial columns data_geo and the other data.

```python
# Columns for `geo_data`, stored in `geo_columns`
spatial_columns = ['road_name', 'UTM32_north', 'UTM32_east']

# Columns for `data`, stored in `select_columns`
hours = ['kl.{}-{}'.format(str(h).zfill(2), str(h+1).zfill(2)) for h in range(24)]
select_columns = ['road_name', 'date'] + hours
```

In [ ]:
# [Answer to Ex. 7.2.11]

> **Ex. 7.2.12:** Drop the duplicate rows in data_geo.

In [ ]:
# [Answer to Ex. 7.2.12]

**Formatting: wide and narrow format**

When talking about two-dimensional data (matrices, tables or dataframes, we can call it many things), we can either say that it is in wide or long format (see explanation here, "wide" and "long" are used interchangably). In Pandas we can use the commands stack and unstack to move between these formats.

The wide format has the advantage that it often requires less storage and is easier to read when printed. On the other hand the long format can be easier for modelling, because each observation has its own row. Turns out that the latter is what we most often need.

> **Ex. 7.2.13:** Turn the dataset from wide to long so hourly data is now vertically stacked. Store this dataset in a dataframe called data. Name the column with hourly information hour_period. 

> _Hint:_ pandas' melt function may be of use.

In [ ]:
# [Answer to Ex. 7.2.13]

**Categorical data**

Categorical data can contain Python objects, usually strings. These are smart if you have variables with string observations that are long and often repeated, e.g. with road names.

> **Ex. 7.2.14:** Use the `.astype` method to convert the type of the road_name column to categorical.

In [ ]:
# [Answer to Ex. 7.2.14]

### Structure temporal data

Pandas has native support for working with temporal data. This is handy as much 'big data' often has time stamps which we can make Pandas aware of. Once we have encoded temporal data it can be used to extract information such as the hour, second etc.

> **Ex. 7.2.15:** Create a new column called hour which contains the hour-of-day for each row. 

In [ ]:
# [Answer to Ex. 7.2.15]

>**Ex. 7.2.16:** Create a new column called time, that contains the time of the row in datetime format. Delete the old temporal columns (hour, hour_period, date) to save memory.

> _Hint:_ try making an intermediary series of strings that has all temporal information for the row; then use pandas to_datetime function where you can specify the format of the date string.

In [ ]:
# [Answer to Ex. 7.2.16]

> **Ex. 7.2.17:** Using your time column make a new column called weekday which stores the weekday (in values between 0 and 6) of the corresponding datetime.

> _Hint:_ try using the dt method for the series called time; dt has some relevant methods itself.

In [ ]:
# [Answer to Ex. 7.2.17]

### Statistical descriptions of traffic data

> **Ex. 7.2.18:** Print the "descriptive statistics" of the traffic column. Also show a kernel density estimate of the values.

> _Hint:_ Use the describe method of pandas dataframes for the first task. Use seaborn for the second. 

In [ ]:
# [Answer to Ex. 7.2.18]

> **Ex. 7.2.19:** Which road has the most average traffic?

In [ ]:
# [Answer to Ex. 7.2.19]

> **Ex. 7.2.20:** Compute annual, average road traffic during day hours (9-17). Which station had the least traffic in 2013? Which station has seen highest growth in traffic from 2013 to 2014?

In [58]:
# [Answer to Ex. 7.2.19]